In [133]:
# part 1
import copy
lines = open("input_20.txt").read().splitlines()
# [print(x) for x in lines]
# print(' ')

# build dictionary of instructions
cfg = {}
start_state = 0  # 0:off, 1:on
for line in lines:
    input, outputs = line.split(' -> ')
    if input[0] in '%&':
        key = input[1:]
        cfg[key] = [start_state, input[0], outputs.split(', ')]
    else:
        key = input
        cfg[key] = [start_state, '', outputs.split(', ')]
cfg_orig = copy.deepcopy(cfg)
# [print(x,cfg[x]) for x in cfg]
# print(' ')

# build a dictionary of conjuction inputs
conj_inputs = {}
for gate in cfg:
    if cfg[gate][1] == '&':
        inputs = []
        for gate2 in cfg:
            if gate in cfg[gate2][2]:
                inputs.append(gate2)
        conj_inputs[gate] = inputs
# print('inputs =',conj_inputs)

# find the no-outputs
no_outputs = set([])
for gate in cfg:
    for output in cfg[gate][2]:
        if output not in cfg:
            no_outputs.add(output)
# print('no_outputs =',no_outputs)


# push the button
pulses = [0,0]  # lows, highs
for i in range(1000):
    gate = 'button'
    pulse = 'L'
    outputs = ['broadcaster']
    pulses[0] += 1
    # print(f'{gate:11} --> {pulse} --> {outputs[0]:20}pulses = {pulses}')

    todo = []
    gate = 'broadcaster'
    pulse = 'L'
    outputs = cfg[gate][2]
    pulses[0] += len(outputs)
    for a in range(len(outputs)):
        todo.append([pulse,outputs[a]])
        # print(f'{gate:11} --> {pulse} --> {outputs[a]:20}pulses = {pulses}')

    while len(todo) > 0:
        # print(todo)
        pulse,gate = todo.pop(0)
        if gate in cfg:
            state = cfg[gate][0]
            type = cfg[gate][1]
            outputs = cfg[gate][2]
        else:
            # this gate is not in the configuration, probably the output
            state = pulse
            type = '-'
            outputs = 'none'
            # print('unconfigured gate:',gate,'is in state:',state)
        if pulse == 'H' and type == '%':  # high pulse to flip-flop
            1  # do nothing
        elif pulse == 'L' and type == '%':  # low pulse to flip-flop
            if state == 0:
                cfg[gate][0] = 1
                pulse = 'H'
                pulses[1] += len(outputs)
            else:
                cfg[gate][0] = 0
                pulse = 'L'
                pulses[0] += len(outputs)

            for a in range(len(outputs)):
                todo.append([pulse,outputs[a]])
                # print(f'{gate:11} --> {pulse} --> {outputs[a]:20}pulses = {pulses}')

        # Conjuction: If it remembers high pulses for all inputs, it sends a low pulse;
        # otherwise, it sends a high pulse.
        if type == '&':  # conjunction
            inputs = conj_inputs[gate]
            states = [cfg[input][0] for input in inputs]
            if states == [1]*len(states):  # all input states are high
                cfg[gate][0] = 0
                pulse = 'L'
                pulses[0] += len(outputs)
            else:
                cfg[gate][0] = 1
                pulse = 'H'
                pulses[1] += len(outputs)

            for a in range(len(outputs)):
                todo.append([pulse,outputs[a]])
                # print(f'{gate:11} --> {pulse} --> {outputs[a]:20}pulses = {pulses}')

    # print(f'button push #: {i+1}, config the same? {cfg == cfg_orig}')
    # print(cfg_orig)
    # print(cfg)
    # print('i =',i,'pulses =',pulses,'lows*highs =',pulses[0]*pulses[1])
    # print(' ')
    if cfg == cfg_orig:
        print(f'button pushed: {i+1}x, config the same? {cfg == cfg_orig}')
        break

print('i =',i,'pulses =',pulses,'lows*highs =',pulses[0]*pulses[1])


i = 999 pulses = [16195, 42243] lows*highs = 684125385


In [171]:
# part 2
from math import lcm
import copy
lines = open("input_20.txt").read().splitlines()
# [print(x) for x in lines]
# print(' ')

# build dictionary of instructions
cfg = {}
start_state = 0  # 0:off, 1:on
for line in lines:
    input, outputs = line.split(' -> ')
    if input[0] in '%&':
        key = input[1:]
        cfg[key] = [start_state, input[0], outputs.split(', ')]
    else:
        key = input
        cfg[key] = [start_state, '', outputs.split(', ')]
cfg_orig = copy.deepcopy(cfg)
# [print(x,cfg[x]) for x in cfg]
# print(' ')

# build a dictionary of conjuction inputs
conj_inputs = {}
flop_gates = []
for gate in cfg:
    if cfg[gate][1] == '&':
        inputs = []
        for gate2 in cfg:
            if gate in cfg[gate2][2]:
                inputs.append(gate2)
        conj_inputs[gate] = inputs
    else:
        flop_gates.append(gate)
# print('inputs =',conj_inputs)
        
# find the no-outputs
no_outputs = set([])
for gate in cfg:
    for output in cfg[gate][2]:
        if output not in cfg:
            no_outputs.add(output)
# print('no_outputs =',no_outputs)


# push the button
pulses = [0,0]  # lows, highs
hb_repeaters = [0]*4
for i in range(5000):
    button_pushed = True
    gate = 'button'
    pulse = 'L'
    outputs = ['broadcaster']
    pulses[0] += 1
    # print(f'{gate:11} --> {pulse} --> {outputs[0]:20}pulses = {pulses}')

    todo = []
    gate = 'broadcaster'
    pulse = 'L'
    outputs = cfg[gate][2]
    pulses[0] += len(outputs)
    for a in range(len(outputs)):
        todo.append([pulse,outputs[a]])
        # print(f'{gate:11} --> {pulse} --> {outputs[a]:20}pulses = {pulses}')

    while len(todo) > 0:
        # print(todo)
        pulse,gate = todo.pop(0)
        if gate in cfg:
            state = cfg[gate][0]
            type = cfg[gate][1]
            outputs = cfg[gate][2]
        else:
            # this gate is not in the configuration, probably the output
            state = pulse
            type = '-'
            outputs = 'none'
            # print('unconfigured gate:',gate,'is in state:',state)
        if pulse == 'H' and type == '%':  # high pulse to flip-flop
            1  # do nothing
        elif pulse == 'L' and type == '%':  # low pulse to flip-flop
            if state == 0:
                cfg[gate][0] = 1
                pulse = 'H'
                pulses[1] += len(outputs)
            else:
                cfg[gate][0] = 0
                pulse = 'L'
                pulses[0] += len(outputs)

            for a in range(len(outputs)):
                todo.append([pulse,outputs[a]])
                # print(f'{gate:11} --> {pulse} --> {outputs[a]:20}pulses = {pulses}')

        # Conjuction: If it remembers high pulses for all inputs, it sends a low pulse;
        # otherwise, it sends a high pulse.
        if type == '&':  # conjunction
            inputs = conj_inputs[gate]
            states = [cfg[input][0] for input in inputs]
            if states == [1]*len(states):  # all input states are high
                cfg[gate][0] = 0
                pulse = 'L'
                pulses[0] += len(outputs)
            else:
                cfg[gate][0] = 1
                pulse = 'H'
                pulses[1] += len(outputs)

            for a in range(len(outputs)):
                todo.append([pulse,outputs[a]])
                # print(f'{gate:11} --> {pulse} --> {outputs[a]:20}pulses = {pulses}')


        # inputs to rx is &hb
        # inputs to &hb are &['js', 'zb', 'bs', 'rr']
        hb_inputs = ['js', 'zb', 'bs', 'rr']
        hb_input_states = [cfg[x][0] for x in hb_inputs]
        if sum(hb_input_states) > 0 and button_pushed is True:
            button_pushed = False
            for a in range(4):
                if hb_input_states[a] == 1:
                    hb_repeaters[a] = i+1
            print('i = ',i,'hb_inputs: ',hb_input_states, 'hb_repeater: ',hb_repeaters)
            print('lcm repeaters: ',lcm(hb_repeaters[0],hb_repeaters[1],hb_repeaters[2],hb_repeaters[3]))

        if 'rx' in outputs and pulse == 'L':
            print('i = ',i, gate,'sent',pulse,'pulse delivered to ',outputs)
        
    # print(f'button push #: {i+1}, config the same? {cfg == cfg_orig}')
    # print(cfg_orig)
    # print(cfg)
    # print('i =',i,'pulses =',pulses,'lows*highs =',pulses[0]*pulses[1])
    # print(' ')
            
    # flop states
    flop_states = [0]*len(flop_gates)
    for a, gate in enumerate(flop_gates):
        flop_states[a] = cfg[gate][0]

    if sum(flop_states) == 0:
        print(f'button pushed: {i+1}x, flops are 0')
        break    

    # if (i+1) % 1000 == 0:
    #     print('i =',i,'pulses =',pulses,'lows*highs =',pulses[0]*pulses[1])

# print('i =',i,'pulses =',pulses,'lows*highs =',pulses[0]*pulses[1])


i =  3732 hb_inputs:  [0, 0, 0, 1] hb_repeater:  [0, 0, 0, 3733]
lcm repeaters:  0
i =  3760 hb_inputs:  [1, 0, 0, 0] hb_repeater:  [3761, 0, 0, 3733]
lcm repeaters:  0
i =  4000 hb_inputs:  [0, 0, 1, 0] hb_repeater:  [3761, 0, 4001, 3733]
lcm repeaters:  0
i =  4020 hb_inputs:  [0, 1, 0, 0] hb_repeater:  [3761, 4021, 4001, 3733]
lcm repeaters:  225872806380073
